In [1]:
import tensorflow as tf
import pandas as pd
import os
import random
import numpy as np

## Import Data ##

In [31]:
# import training dataset
root_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
data_path = '/data/'
filename = 'input_file.pkl'

In [32]:
# import sampled training dataset
df_samp = pd.read_pickle(root_path + data_path + filename)
# remove unnecessary columns
df_samp.drop(['visitor_id','session_id','visit_len','pageviews','cust_serv_visit','date'], axis=1, inplace=True)
df_samp.is_customer.replace(to_replace=-1, value=0, inplace=True)
df_samp = df_samp.astype('float32')

In [33]:
df_samp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89862 entries, 1256 to 89860
Data columns (total 10 columns):
checkout_visit       89862 non-null float32
source_id            89862 non-null float32
device_id            89862 non-null float32
country_id           89862 non-null float32
city_id              89862 non-null float32
nb_past_visits       89862 non-null float32
is_customer          89862 non-null float32
nb_past_checkouts    89862 non-null float32
last_pageviews       89862 non-null float32
last_visit_len       89862 non-null float32
dtypes: float32(10)
memory usage: 4.1 MB


## Deep Feed-Forward Neural Net ##

Code borrowed from :https://github.com/aymericdamien/TensorFlow-Examples/blob/master/examples/3_NeuralNetworks/multilayer_perceptron.py

In [34]:
# Parameters
learning_rate = 0.1
training_epochs = 100
batch_size = 100
display_step = 10

# Network Parameters
n_hidden_1 = 20 # 1st layer number of features
n_hidden_2 = 20 # 2nd layer number of features
n_input = 9 # MNIST data input (img shape: 28*28)
n_classes = 2 # MNIST total classes (0-9 digits)

# define data placeholders
x = tf.placeholder("float", shape=[None, n_input])
y = tf.placeholder("float", shape=[None, n_classes])

In [42]:
# Create model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activation
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    
    # Hidden layer with RELU activation
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    
    # Output layer with linear activation
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

In [36]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

In [37]:
# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [38]:
# Initializing the variables
init = tf.global_variables_initializer()

In [39]:
# Launch the graph
# create interactive session
sess = tf.InteractiveSession()
# initialize variables
sess.run(init)
# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.        
    total_batch = int(len(df_samp)/batch_size)
    # log results for tensorboard
#     writer = tf.summary.FileWriter("./logs/nn_logs", sess.graph) # for 0.8
#     merged = tf.summary.merge_all()
    # Loop over all batches
    for i in range(total_batch):
        df_tmp = df_samp
        batch = df_tmp.loc[random.sample(df_tmp.index.values.tolist(), batch_size),:]
        target_samp = batch.pop('checkout_visit')
        y_tp = target_samp.values.reshape(-1,1)
        batch_y = np.concatenate((y_tp, np.abs(y_tp-1)), axis=1)
        batch_y.shape
        batch_x = batch.values
        batch_x.shape
        # Run optimization op (backprop) and cost op (to get loss value)
        summary, c = sess.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
        # Write summary
        writer.add_summary(summary, epoch)
        # Compute average loss
        avg_cost += c / total_batch
    # Display logs per epoch step
    if epoch % display_step == 0:
        print("Epoch:", '%04d' % (epoch+1), "cost=", \
            "{:.9f}".format(avg_cost))
print("Optimization Finished!")

Epoch: 0001 cost= 25.485970318
Epoch: 0011 cost= 0.695257530
Epoch: 0021 cost= 0.695403543
Epoch: 0031 cost= 0.695226603
Epoch: 0041 cost= 0.695179830
Epoch: 0051 cost= 0.694769020
Epoch: 0061 cost= 0.695091939
Epoch: 0071 cost= 0.695297793
Epoch: 0081 cost= 0.695540762
Epoch: 0091 cost= 0.694916498
Optimization Finished!


In [40]:
# prepare dataset
df_tmp = df_samp
target = df_tmp.pop('checkout_visit')
y_tp = target.values.reshape(-1,1)
y_all = np.concatenate((y_tp, np.abs(y_tp-1)), axis=1)
x_all = df_tmp.values

In [41]:
# Test model
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
# Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
acc = accuracy.eval(feed_dict={x: x_all, y: y_all}) * 100
print('Accuracy: {0:2} %'.format(acc))

Accuracy: 50.01558065414429 %
